<a href="https://colab.research.google.com/github/jdelagarzaf/TC2008B.103/blob/main/M1_Actividad_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jorge Adrian de la Garaz Flores - A00838816
19 de Agosto, 2025

Este programa crea un agente que se mueve de manera espontanea y por donde pasa limpia las celdas que esten sucias. Las celdas sucias tambien son determinadas de manera espontanea, pero su cantidad esta definida por un porcentaje de la cantidad total de celdas.
La cantidad de celdas y el tiempo de ejecución pueden ser controlados como variables del sistema.

Este agente no es muy eficiente, ya que recorre caminos por donde ya cruzo, y depende de la suerte para poder limpiar las celdas. Un mejor agente recorreria todas las celdas, o mejor aun, se dirigeria exclusivamente a las celdas sucias mas cercanas, o recorrería el camino corto para limpiar todas las celdas sucias.

Otra manera de hacer algo similar es mediante Mesa, una libreria open source tambien para Python, y tiene muchas herramientas y add-ons. Otra opcion buena es NetLogo, aunque esta ya no es dentro de Python y tiene su propio lenguaje de programación, pero permite integración web. Otro muy famoso de python es SymPy.

In [ ]:
!pip install agentpy

In [ ]:
#Se importan todas las librerias
import agentpy as ap
import numpy as np
from IPython.display import HTML
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import random

# Variables iniciales.
# M y N definen el tamaño de la cuadricula
# Sucio es el porcentaje de celdas sucidas
# Tiempo es el maximo tiempo de ejecución, o mas bien, el maximo numero de pasos
M = 5
N = 4
Sucio = 0.15
Tiempo = 15

In [ ]:
# Se crea el Dummy Agent en la posicion incial de 0,0
class DummyAgent(ap.Agent):
  def setup(self):
    self.pos = (0,0)

  def execute(self):
    # Se saca la posición del agente
    posX, posY = self.model.enviroment.positions[self]

    # Se calcula un paso aleatorio en X y Y
    cambioX = random.randint(-1, 1)
    cambioY = random.randint(-1, 1)

    # Se camina a esa nueva posición.
    # Se usa la combinación min para que no salga del area de juego
    nuevaX = max(0, min(M-1, posX + cambioX))
    nuevaY = max(0, min(N-1, posY + cambioY))

    # Se mueve el agente en la cuadricula
    self.model.enviroment.move_to(self, (nuevaX, nuevaY))

# Se crea el Dummy Model del juego
class DummyModel(ap.Model):
  def setup(self):
    # Se crea un espacio M x N con un agente en la posición 0,0
    self.enviroment = ap.Grid(self,(M, N))
    self.agent = DummyAgent(self)
    self.agent.setup()
    self.enviroment.add_agents([self.agent], positions=[(0, 0)])

    # Se crean las celdas sucias con el apoyo de una matriz
    num_sucias = round(Sucio * M*N)
    self.dirt = np.zeros((M, N), dtype=bool)

    # Se usa una matriz para guardar las posiciones de las celdas sucias
    idxs = np.random.choice(M*N, size=num_sucias, replace=False)
    rows = idxs // N
    cols = idxs % N
    self.dirt[rows, cols] = True

  def step(self):
    # Se ejecuta el paso del agente con cada frame
    self.agent.execute()

    # Se saca la posición y se checa si esta sucia
    rowActual, colActual = self.enviroment.positions[self.agent]
    if self.dirt[rowActual, colActual]:
      self.dirt[rowActual, colActual] = False

In [ ]:
def my_plot(model, ax):
    # Se crea el grafico con la capa de tierra
    ax.clear()
    dirt_layer = model.dirt.astype(int)
    view_grid = dirt_layer.copy()

    # Se obtiene la posición del agente
    (posX, posY) = model.enviroment.positions[model.agent]
    view_grid[posX, posY] = 2

    # Se colorea cafe si esta sucio, gris si es el agente, o blanco
    cmap = ListedColormap([ (1.0, 1.0, 1.0, 1.0), (0.59, 0.29, 0.0, 1.0), (0.2, 0.2, 0.2, 1.0) ])
    ax.imshow(view_grid, cmap=cmap, vmin=0, vmax=2, origin='upper')

    # Se ajustan las dimensiones y la cuadricula.
    ax.set_xlim(0, N-0.5)
    ax.set_ylim(0, M-0.5)
    ax.set_xticks(np.arange(-0.5, N, 1))
    ax.set_yticks(np.arange(-0.5, M, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True, linewidth=0.5)

    # Se ponen unos datos arriba para hacer tracking de cuanto ha limpiado
    ax.set_title(f"Cuadricula: {M}x{N} | Sucias: {np.count_nonzero(model.dirt)}")

In [ ]:
# Se ejecuta todo y se dibuja
fig, ax = plt.subplots()
parameters = {'print': False, 'steps': Tiempo}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())